# あなたの最初の課題へようこそ！

指示は以下にあります。これを試してみてください、そしてあなたが行き詰まったらソリューションフォルダーを見てください（または私に聞いてください！）

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">割り当てに到達する直前 --</h2>
            <span style="color:#f71;">コースの有用なリソースのこのページであなたを指すのに少し時間がかかると思いました。これには、すべてのスライドへのリンクが含まれます。<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            これをブックマークしておくと、時間の経過とともにもっと便利なリンクを追加し続けます。
            </span>
        </td>
    </tr>
</table>

# 宿題の運動の割り当て

Day 1プロジェクトをアップグレードして、Webページを要約して、OpenaiではなくOllamaを介してローカルに実行されるオープンソースモデルを使用します

有料APIを使用したくない場合は、このテクニックをすべての後続のプロジェクトに使用できます。

**利点：**
1. API料金なし - オープンソース
2。データは箱を離れません

**短所：**
1.フロンティアモデルよりも大幅に少ないパワー

## オラマのインストールに関する要約

[ollama.com](https://ollama.com)にアクセスしてインストールしてください！

完了すると、Ollamaサーバーはすでにローカルで実行されているはずです。  
訪問する場合：  
[http://localhost:11434/](http://localhost:11434/)

「Ollamaが実行されている」というメッセージが表示されます。  

そうでない場合は、新しいターミナル（MAC）またはPowerShell（Windows）を持ち上げて、「Ollama Serve」を入力します  
別の端末（MAC）またはPowerShell（Windows）で、「Ollama Pull llama3.2」を入力します  
次に、[http://localhost:11434/](http://localhost:11434/)をもう一度試してください。

オラマがマシンで遅い場合は、代替として「llama3.2：1b」を使用してみてください。端子またはPowerShellからllama3.2：1b`を "ollama pull llama3.2：1b`を実行し、以下のコードを` model = "llama3.2" `から` model = "llama3.2：1b" `から変更します。

In [ ]:
# 輸入

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [ ]:
# 定数

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [ ]:
# Openaiに使用したのと同じ形式を使用してメッセージリストを作成する

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [ ]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [ ]:
# モデルがロードされていることを確認しましょう

!ollama pull llama3.2

In [ ]:
# これが何らかの理由で機能しない場合は、次のセルの2つのバージョンを試してください
# このラボの上部にある「オラマのインストールに関する要約」の指示を再確認してください
# そして、それがうまくいかなければ - 私に連絡してください！

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

# Ollamaパッケージの紹介

そして今、私たちは同じことをしますが、直接HTTP呼び出しの代わりにエレガントなOllama Pythonパッケージを使用します。

ボンネットの下では、localhost：11434で実行されているOllamaサーバーに上記と同じ呼び出しを行っています

In [ ]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

## 代替アプローチ -  Openai Pythonライブラリを使用してOllamaに接続する

In [ ]:
# 実際には、一部の人々が好むかもしれない代替アプローチがあります
# OpenaiクライアントPythonライブラリを使用して、Ollamaを呼び出すことができます。

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

## なぜそれが機能するのか混乱していますか？

奇妙に思えますよね？ Openaiコードを使用してOllamaを呼び出しましたか？どうしたの？！

これがスクープです：

Pythonクラス「Openai」は、インターネットを介してエンドポイントに電話をかけるOpenaiエンジニアによって書かれた単純なコードです。  

`openai.chat.completions.create（）`を呼び出すと、このPythonコードは次のURLにWeb要求を行うだけです。

このようなコードは「クライアントライブラリ」と呼ばれます。これは、Webリクエストを行うためにマシンで実行されるラッパーコードです。 GPTの実際のパワーは、コンピューターではなく、このAPIの背後にあるOpenaiのクラウドで実行されています！

Openaiは非常に人気があったため、他の多くのAIプロバイダーが同一のWebエンドポイントを提供していたため、同じアプローチを使用できます。

したがって、Ollamaにはhttp：// localhost：11434/v1/chat/completionsのローカルボックスで実行されているエンドポイントがあります  
そして、2週目には、ジェミニやディープシークなど、他の多くのプロバイダーもこれを行っていることがわかります。

そして、OpenAIのチームは素晴らしいアイデアを持っていました。彼らはクライアントライブラリを拡張して、別の「ベースURL」を指定し、ライブラリを使用して互換性のあるAPIを呼び出すことができます。

それでおしまい！

あなたが言うとき： `ollama_via_openai = openai（base_url = 'http：// localhost：11434/v1'、api_key = 'ollama'）`  
その後、これは同じエンドポイント呼び出しを行いますが、Openaiの代わりにOllamaに。

## また、驚くべき推論モデルDeepseekを試してみてください

ここでは、1.5Bに蒸留されているDeepSeek-Reasonerのバージョンを使用します。  
これは実際には、deepseek R1によって生成された同期データを使用して微調整されたQwenの1.5bバリアントです。

DeepSeekのその他のサイズは、[here](https://ollama.com/library/deepseek-r1)であり、671bパラメーターバージョンの完全なバージョンまでです。

In [ ]:
!ollama pull deepseek-r1:1.5b

In [ ]:
# これには数分かかる場合があります！その後、<shink>タグ内に魅力的な「思考」トレースが表示され、その後にいくつかの適切な定義が表示されます

response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:1.5b",
    messages=[{"role": "user", "content": "Please give definitions of some core concepts behind LLMs: a neural network, attention and the transformer"}]
)

print(response.choices[0].message.content)

# 今あなたのための運動

Day1からコードを取り、ここに組み込んで、Openaiの代わりにローカルで実行されているLlama 3.2を使用するWebサイトsummarizerを構築します。上記のアプローチのいずれかを使用します。